# AI Beast - Model Management

This notebook demonstrates model management operations with AI Beast.

## Features Covered
- Scanning local models
- Listing Ollama models
- Storage analysis
- Model metadata inspection

In [ ]:
# Add AI Beast workspace to path
import sys
sys.path.insert(0, '/home/jovyan/ai_beast')

# Standard imports
import os
import asyncio
from pathlib import Path
from pprint import pprint

In [ ]:
# Initialize AI Beast context
from modules.container import AppContext

# Create context pointing to workspace
context = AppContext(
    base_dir=Path('/home/jovyan/ai_beast'),
    models_dir=Path('/home/jovyan/models'),
    data_dir=Path('/home/jovyan/work'),
    ports={},
    features={},
)

print(f"Base Dir: {context.base_dir}")
print(f"Models Dir: {context.models_dir}")
print(f"Data Dir: {context.data_dir}")

## Scan Local Models

Scan all local model directories for GGUF, safetensors, and other model files.

In [ ]:
from modules.llm.manager_async import AsyncLLMManager

async def scan_models():
    """Scan and list all local models."""
    async with AsyncLLMManager(context) as manager:
        models = await manager.scan_local_models_async(force=True)
        return models

# Run scan
models = await scan_models()

print(f"Found {len(models)} models:\n")
for model in models:
    print(f"📦 {model.name}")
    print(f"   Location: {model.location.value}")
    print(f"   Size: {model.size_human}")
    print(f"   Type: {model.model_type}")
    if model.quantization:
        print(f"   Quantization: {model.quantization}")
    print()

## Ollama Models

List models available through the Ollama API.

In [ ]:
from modules.ollama import OllamaClient

async def list_ollama_models():
    """List Ollama models."""
    # Connect to Ollama on host
    async with OllamaClient(base_url="http://host.docker.internal:11434") as client:
        return await client.list_models()

try:
    ollama_models = await list_ollama_models()
    print(f"Ollama models: {len(ollama_models)}\n")
    for model in ollama_models:
        print(f"🤖 {model['name']}")
        if 'size' in model:
            size_gb = model['size'] / (1024**3)
            print(f"   Size: {size_gb:.2f} GB")
except Exception as e:
    print(f"❌ Could not connect to Ollama: {e}")
    print("Make sure Ollama is running on the host machine.")

## Storage Analysis

Analyze model storage usage with visualizations.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Convert models to DataFrame
if models:
    df = pd.DataFrame([{
        'name': m.name,
        'location': m.location.value,
        'size_bytes': m.size_bytes,
        'size_gb': m.size_bytes / (1024**3),
        'model_type': m.model_type,
        'quantization': m.quantization or 'N/A',
    } for m in models])
    
    display(df)
else:
    print("No models found to analyze.")

In [ ]:
# Storage by location visualization
if models:
    storage_by_location = df.groupby('location')['size_gb'].sum().sort_values(ascending=False)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Bar chart
    storage_by_location.plot(kind='bar', ax=axes[0], color='steelblue')
    axes[0].set_title('Model Storage by Location')
    axes[0].set_ylabel('Size (GB)')
    axes[0].set_xlabel('Location')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Pie chart
    storage_by_location.plot(kind='pie', ax=axes[1], autopct='%1.1f%%')
    axes[1].set_title('Storage Distribution')
    axes[1].set_ylabel('')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Total storage: {df['size_gb'].sum():.2f} GB")

In [ ]:
# Storage by model type
if models and 'model_type' in df.columns:
    storage_by_type = df.groupby('model_type')['size_gb'].sum().sort_values(ascending=False)
    
    plt.figure(figsize=(10, 5))
    storage_by_type.plot(kind='barh', color='coral')
    plt.title('Storage by Model Type')
    plt.xlabel('Size (GB)')
    plt.ylabel('Model Type')
    plt.tight_layout()
    plt.show()

## Model Details

Inspect detailed metadata for a specific model.

In [ ]:
# Pick the largest model for inspection
if models:
    largest_model = max(models, key=lambda m: m.size_bytes)
    print(f"Largest Model: {largest_model.name}")
    print(f"="*50)
    pprint(largest_model.to_dict())